# UWB 자율주행 물류 로봇 통합 관리 시스템

In [1]:
robot_list = [[1, 'locus_bot.jpg', 'Locus_bot', '현재 로봇 6대 운행중', '없음'],
                 [2, 'chuck.jpg', 'Chuck', '현재 로봇 6대 운행중', '없음'],
                 [3, 'go_cart_250.jpg', 'gocart_2500', '현재 로봇 6대 운행중', '없음'],
                 [4, 'M3_1000.jpg', 'M3_1000', '현재 로봇 6대 운행중', '없음'],
                 [5, 'hanhaw_logistics.jpg', 'hanhaw_logistics', '현재 로봇 6대 운행중', '없음'],
                 [6, 'TES.jpg', 'TES', '현재 로봇 6대 운행중', '없음']
             ]
warehouse_list = [[i, f'warehouse0{i}.png', f'창고명{i}', '현재 로봇 6대 운행중', '없음'] for i in range(1, 6)]
w_info = [[i, f'warehouse0{i}.png', f'창고명{i}','현재 로봇 10대 운행중', '없음', 
          {'Locus_bot':[i*6-5, '작업중'],
           'Chuck':[i*6-2, '작업중'],
           'gocart_2500' :[i*6-3, '대기중'],
           'M3_1000' : [i*6, '작업중'],
           'hanhaw_logistics' :[i*6-1, '대기중'],
           'TES':[i*6-4, '점검중']
          }] for i in range(1, 6)]
r_info = {'Locus_bot': [[i, '작업중', f'w_{i*3-2}'] for i in range(1, 7)],
           'Chuck':[[i, '대기중', f'w_{i*3-2}'] for i in range(7, 13)],
           'gocart_2500' :[[i, '작업중', f'w_{i*3-2}'] for i in range(13,19)],
           'M3_1000' : [[i, '점검중', f'w_{i*3-2}'] for i in range(19, 25)],
           'hanhaw_logistics' :[[i, '작업중', f'w_{i*3-2}'] for i in range(25, 31)],
           'TES':[[31, '작업중', 1,(305, 400)], [32, '대기중', 3,(320, 240)], [33, '점검중', 2,(25, 38)], [34, '작업중', 1,(325, 120)], [35, '작업중', 1,(5, 400)], [36, '대기중', 2,(10, 500)]]
          }

In [ ]:
from flask import Flask, session, render_template, redirect, request, url_for
from markupsafe import escape
from datetime import timedelta, datetime
import random

app = Flask(__name__)
app.secret_key = f"I'm_on_TOP!{random.randint(0, 99999):0>5}"
app.config["PERMANENT_SESSION_LIFETIME"] = timedelta(minutes=60) # 로그인 지속 시간

token, info, cage_list, cage_lifo = '', '', '', ''

######### 홈 #########
@app.route('/') # home 
def index() :
    return render_template('home.html', session=session, title="홈")


######### 창고 #########
@app.route('/warehouse')
def warehouse() :
    
    warehouse_robot = {'what' : '창고',
                      'list' : warehouse_list,
                      'image' : 'warehouse.png'}
    return render_template('warehouse_robot.html', session=session, warehouse_robot=warehouse_robot)

# 창고 정보 확인
@app.route('/warehouse/w_<w_no>')
def warehouse_show(w_no) :
    for w in w_info :
        if w[0] == int(w_no) :
            return render_template('warehouse_show.html', session=session, w_state=w)

######### 로봇 #########
@app.route('/robot')
def robot() :
    warehouse_robot = {'what' : '로봇',
                      'list' : robot_list,
                      'image' : 'robot.png'}
    return render_template('warehouse_robot.html', session=session, warehouse_robot=warehouse_robot)

# 위치 확인- 첫화면
@app.route('/robot/<r_model>')
def robot_loc0(r_model) :
    return render_template('robot_loc.html', session=session, r_name=r_model, r_list=r_info[r_model], r_state=None)


# 위치 확인 - 모델 번호로 검색
@app.route('/robot/<r_model>&<r_no>')
def robot_loc(r_model,r_no) :
    for r in r_info[r_model] :
        if r[0] == int(r_no) :
            return render_template('robot_loc.html', session=session, r_name=robot_list[int(r_no)][2], r_list=r_info[r_model], r_state=r)
#     return redirect(url_for())

######### 직원 #########
@app.route('/employee')
def employee() :
    return render_template('employee.html', session=session)

if __name__ == '__main__' :
    app.run(host="192.168.20.153", port=2048)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://192.168.20.153:2048
Press CTRL+C to quit
192.168.20.153 - - [02/Nov/2022 11:14:28] "GET / HTTP/1.1" 200 -
192.168.20.153 - - [02/Nov/2022 11:14:28] "GET /static/bootstrap.min.css HTTP/1.1" 200 -
192.168.20.153 - - [02/Nov/2022 11:14:28] "GET /static/bootstrap.css HTTP/1.1" 200 -
192.168.20.153 - - [02/Nov/2022 11:14:28] "GET /static/common.css HTTP/1.1" 200 -
192.168.20.153 - - [02/Nov/2022 11:14:28] "GET /static/footer.css HTTP/1.1" 200 -
192.168.20.153 - - [02/Nov/2022 11:14:28] "GET /static/common.js HTTP/1.1" 200 -
192.168.20.153 - - [02/Nov/2022 11:14:28] "GET /static/bootstrap.bundle.min.js HTTP/1.1" 200 -
192.168.20.153 - - [02/Nov/2022 11:14:28] "GET /static/prism.js HTTP/1.1" 200 -
192.168.20.153 - - [02/Nov/2022 11:14:28] "GET /static/sweetalert.min.js HTTP/1.1" 200 -
192.168.20.153 - - [02/Nov/2022 11:14:28] "GET /static/custom.js HTTP/1.1" 404 -
192.168.20.153 - - [02/Nov/2022 11:14:29] "GET /static/img/logo.png HTTP/1.1" 200 -
192.168.20.153 - - [02/No

192.168.20.153 - - [02/Nov/2022 11:15:15] "GET /static/footer.css HTTP/1.1" 304 -
192.168.20.153 - - [02/Nov/2022 11:15:15] "GET /static/bootstrap.min.css HTTP/1.1" 304 -
192.168.20.153 - - [02/Nov/2022 11:15:15] "GET /static/custom.js HTTP/1.1" 404 -
192.168.20.153 - - [02/Nov/2022 11:15:15] "GET /static/common.css HTTP/1.1" 304 -
192.168.20.153 - - [02/Nov/2022 11:15:15] "GET /static/common.js HTTP/1.1" 304 -
192.168.20.153 - - [02/Nov/2022 11:15:15] "GET /static/bootstrap.bundle.min.js HTTP/1.1" 304 -
192.168.20.153 - - [02/Nov/2022 11:15:15] "GET /static/prism.js HTTP/1.1" 304 -
192.168.20.153 - - [02/Nov/2022 11:15:15] "GET /static/sweetalert.min.js HTTP/1.1" 304 -
192.168.20.153 - - [02/Nov/2022 11:15:15] "GET /static/img/warehouse01.png HTTP/1.1" 304 -
192.168.20.153 - - [02/Nov/2022 11:15:15] "GET /static/img/logo.png HTTP/1.1" 304 -
192.168.20.153 - - [02/Nov/2022 11:15:15] "GET /static/img/map.png HTTP/1.1" 304 -
192.168.20.153 - - [02/Nov/2022 11:15:15] "GET /static/img/tel.

192.168.20.153 - - [02/Nov/2022 11:15:45] "GET /static/sweetalert.min.js HTTP/1.1" 304 -
192.168.20.153 - - [02/Nov/2022 11:15:45] "GET /static/custom.js HTTP/1.1" 404 -
192.168.20.153 - - [02/Nov/2022 11:15:45] "GET /static/img/warehouse01.png HTTP/1.1" 304 -
192.168.20.153 - - [02/Nov/2022 11:15:45] "GET /static/img/warehouse04.png HTTP/1.1" 304 -
192.168.20.153 - - [02/Nov/2022 11:15:45] "GET /static/img/warehouse02.png HTTP/1.1" 304 -
192.168.20.153 - - [02/Nov/2022 11:15:45] "GET /static/img/logo.png HTTP/1.1" 304 -
192.168.20.153 - - [02/Nov/2022 11:15:45] "GET /static/img/warehouse03.png HTTP/1.1" 304 -
192.168.20.153 - - [02/Nov/2022 11:15:45] "GET /static/img/warehouse05.png HTTP/1.1" 304 -
192.168.20.153 - - [02/Nov/2022 11:15:45] "GET /static/img/warehouse.png HTTP/1.1" 304 -
192.168.20.153 - - [02/Nov/2022 11:15:45] "GET /static/img/map.png HTTP/1.1" 304 -
192.168.20.153 - - [02/Nov/2022 11:15:45] "GET /static/img/tel.png HTTP/1.1" 304 -
192.168.20.153 - - [02/Nov/2022 11:1

192.168.20.153 - - [02/Nov/2022 11:15:52] "GET /static/sweetalert.min.js HTTP/1.1" 304 -
192.168.20.153 - - [02/Nov/2022 11:15:52] "GET /static/img/logo.png HTTP/1.1" 304 -
192.168.20.153 - - [02/Nov/2022 11:15:52] "GET /static/img/warehouse03.png HTTP/1.1" 304 -
192.168.20.153 - - [02/Nov/2022 11:15:52] "GET /static/img/map.png HTTP/1.1" 304 -
192.168.20.153 - - [02/Nov/2022 11:15:52] "GET /static/img/tel.png HTTP/1.1" 304 -
192.168.20.153 - - [02/Nov/2022 11:15:52] "GET /static/img/email.png HTTP/1.1" 304 -
192.168.20.153 - - [02/Nov/2022 11:15:52] "GET /static/contact.js HTTP/1.1" 404 -
192.168.20.153 - - [02/Nov/2022 11:15:56] "GET /warehouse HTTP/1.1" 200 -
192.168.20.153 - - [02/Nov/2022 11:15:56] "GET /static/bootstrap.min.css HTTP/1.1" 304 -
192.168.20.153 - - [02/Nov/2022 11:15:56] "GET /static/bootstrap.css HTTP/1.1" 304 -
192.168.20.153 - - [02/Nov/2022 11:15:56] "GET /static/footer.css HTTP/1.1" 304 -
192.168.20.153 - - [02/Nov/2022 11:15:56] "GET /static/common.css HTTP/1.

192.168.20.153 - - [02/Nov/2022 11:22:51] "GET /static/bootstrap.min.css HTTP/1.1" 304 -
192.168.20.153 - - [02/Nov/2022 11:22:51] "GET /static/common.css HTTP/1.1" 304 -
192.168.20.153 - - [02/Nov/2022 11:22:51] "GET /static/footer.css HTTP/1.1" 304 -
192.168.20.153 - - [02/Nov/2022 11:22:51] "GET /static/bootstrap.css HTTP/1.1" 304 -
192.168.20.153 - - [02/Nov/2022 11:22:51] "GET /static/custom.js HTTP/1.1" 404 -
192.168.20.153 - - [02/Nov/2022 11:22:51] "GET /static/common.js HTTP/1.1" 304 -
192.168.20.153 - - [02/Nov/2022 11:22:51] "GET /static/bootstrap.bundle.min.js HTTP/1.1" 304 -
192.168.20.153 - - [02/Nov/2022 11:22:51] "GET /static/prism.js HTTP/1.1" 304 -
192.168.20.153 - - [02/Nov/2022 11:22:51] "GET /static/sweetalert.min.js HTTP/1.1" 304 -
192.168.20.153 - - [02/Nov/2022 11:22:51] "GET /static/img/logo.png HTTP/1.1" 304 -
192.168.20.153 - - [02/Nov/2022 11:22:51] "GET /static/img/map.png HTTP/1.1" 304 -
192.168.20.153 - - [02/Nov/2022 11:22:51] "GET /static/img/warehouse0